<a href="https://colab.research.google.com/github/AweSamarth/boilerplate-neural-network-sms-text-classifier/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_dataset=pd.read_csv("train-data.tsv",  sep="\t", header=None, names=["hamspam", "message"])
test_dataset=pd.read_csv("valid-data.tsv", sep="\t", header=None, names=["hamspam", "message"])
train_dataset.dropna()
test_dataset.dropna()
train_dataset["hamspam"] = pd.factorize(train_dataset["hamspam"])[0]
test_dataset["hamspam"] = pd.factorize(test_dataset["hamspam"])[0]

print(train_dataset)



In [ ]:
train_labels=train_dataset["hamspam"].values
test_labels=test_dataset["hamspam"].values

train_ds=tf.data.Dataset.from_tensor_slices((train_dataset["message"].values, train_labels))
test_ds= tf.data.Dataset.from_tensor_slices((test_dataset["message"].values, test_labels))



In [ ]:
BATCH_SIZE=32
BUFFER_SIZE=100

In [ ]:
train_ds=train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
print(train_ds)
test_ds=test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
vec=keras.layers.TextVectorization(output_mode='int',
                                   max_tokens=1000,
                                   output_sequence_length=1000)
vec.adapt(train_ds.map(lambda text, label:text))

In [ ]:
vocab= np.array(vec.get_vocabulary())
print(vocab)

In [ ]:
model= keras.Sequential()
model.add(vec)
model.add(keras.layers.Embedding(len(vec.get_vocabulary()), 64,
                                 mask_zero=True))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)))
model.add(keras.layers.Bidirectional(tf.keras.layers.LSTM(32)))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(1))

model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True), optimizer=keras.optimizers.Adam(1e-4), metrics=['accuracy'])
model.summary()



In [ ]:
history = model.fit(train_ds, validation_data=test_ds, validation_steps=30, epochs=10)

In [ ]:
for example, label in train_dataset["hamspam"].take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

In [ ]:
train_dataset=train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset=test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
encoder = keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [ ]:
messages = np.array(messages, dtype="object")
labels=np.array(labels, dtype="object")
labels= keras.utils.to_categorical(labels)
messages= keras.preprocessing.sequence.pad_sequences(messages, maxlen=SEQUENCE_LENGTH)
print(labels)

In [ ]:
print(messages[0])

In [ ]:
model=keras.Sequential()
model.add(keras.layers.Embedding(len(tokenizer.word_index)+1,
                                 EMBEDDING_SIZE
                                 ))
model.add(keras.layers.LSTM(128, recurrent_dropout=0.2))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(2, activation="softmax"))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy",
                  metrics=["accuracy"])
model.build()



model.summary()

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):



  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
